In [69]:
# Loading Libraries
import pandas as pd
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import log_hyperu as hyperu
import tgr as tgr

In [93]:
# Generating the data sets

torch.manual_seed(12345)

# Structure of the data sets
## Set X_A: Sparse Coefficients with many data points
## Set X_B: Dense Coefficients with many data points
## Set X_C: Sparse Coefficients with few data points
## Set X_D: Dense Coefficients with few data points


### Set X_A
variables = 12
sample = 100
true_coefs = torch.tensor([[0],[0],[5.3],[4.2],[0],[0],[6.9],[0],[0],[0],[0],[0]])
X = torch.randn(sample, variables)
Y = X @ true_coefs + torch.randn(sample, 1) * 0.1
print(X)

tensor([[-1.4798,  0.4873, -3.0128,  ...,  1.9991,  0.7848, -1.0195],
        [-0.2106,  0.6268,  0.9318,  ..., -0.5429,  0.4307, -1.9257],
        [ 1.2756, -1.1316,  0.8680,  ..., -0.8407, -0.3963, -0.2591],
        ...,
        [-1.3737,  0.0184,  1.1507,  ..., -0.8902,  0.5128, -0.5483],
        [-0.4525,  1.2544,  1.0292,  ...,  0.3971, -0.7133, -0.6877],
        [-0.3455, -0.0047, -0.2237,  ...,  0.0928,  0.3904,  1.0149]])


In [98]:
trained_model2, coefs, loss_of_optimization = tgr.TripleGammaModel(X, Y, 0.05, 0.75, 0.1, 2, True, num_epochs=2000) # Covariates, Targets, Penalty, a, c, kappa, normalization=True
coefficients = trained_model2.linear.weight.detach().numpy()
#intercept = trained_model2.linear.bias.item()

#print("Coefficients General Function:", coefficients)
#print("Intercept General Function:", intercept)
print(coefficients[0].tolist())

Epoch [100/2000], Loss: 54.46653747558594
Epoch [200/2000], Loss: 42.6019401550293
Epoch [300/2000], Loss: 32.48612976074219
Epoch [400/2000], Loss: 24.077037811279297
Epoch [500/2000], Loss: 17.27841567993164
Epoch [600/2000], Loss: 11.930946350097656
Epoch [700/2000], Loss: 7.806328296661377
Epoch [800/2000], Loss: 4.86722469329834
Epoch [900/2000], Loss: 2.9005184173583984
Epoch [1000/2000], Loss: 1.7011820077896118
Epoch [1100/2000], Loss: 1.1852614879608154
Epoch [1200/2000], Loss: 0.9131882786750793
Epoch [1300/2000], Loss: 0.797982931137085
Epoch [1400/2000], Loss: 0.7394814491271973
Epoch [1500/2000], Loss: 0.7299396991729736
Epoch [1600/2000], Loss: 0.7193539142608643
Epoch [1700/2000], Loss: 0.7230297327041626
Epoch [1800/2000], Loss: 0.7113962173461914
Epoch [1900/2000], Loss: 0.7014321088790894
Epoch [2000/2000], Loss: 0.7086474895477295


TypeError: unsupported format string passed to numpy.ndarray.__format__

In [90]:
#LASSO
# Define your model
class LinearRegressionLasso(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegressionLasso, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

# Define hyperparameters
input_size = 12
output_size = 1
learning_rate = 0.01
num_epochs = 1000
lambda_lasso = 1  # L1 regularization parameter

# Create the model
model = LinearRegressionLasso(input_size, output_size)

# Define loss function (MSE loss with L1 regularization)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X)
    loss = criterion(outputs, Y)
    
    # L1 regularization term
    l1_reg = torch.tensor(0.)
    for param in model.parameters():
        l1_reg += torch.norm(param, p=1)
    
    # Total loss with L1 regularization
    loss += lambda_lasso * l1_reg
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# After training, you can access the learned coefficients
for name, param in model.named_parameters():
    if 'weight' in name:
        print(f'{name}: {param.data}')


Epoch [100/1000], Loss: 19.0645
Epoch [200/1000], Loss: 15.7905
Epoch [300/1000], Loss: 15.5970
Epoch [400/1000], Loss: 15.5760
Epoch [500/1000], Loss: 15.6062
Epoch [600/1000], Loss: 15.5800
Epoch [700/1000], Loss: 15.5879
Epoch [800/1000], Loss: 15.5872
Epoch [900/1000], Loss: 15.5942
Epoch [1000/1000], Loss: 15.5884
linear.weight: tensor([[-6.9520e-03, -7.7376e-03,  4.6682e+00,  3.6015e+00,  7.5944e-03,
         -3.3017e-03,  6.1469e+00, -7.0554e-03,  3.5510e-03, -3.7710e-03,
          1.5011e-03,  2.4193e-03]])
